In [7]:
import numpy as np
import pandas as pd
import recommenders as rec

In [8]:
#df_business=pd.read_feather("../data/business.feather")
df_review=pd.read_feather("../data/review.feather",columns=['business_id','user_id'],use_threads=3)

In [9]:
#print(df_business.head())
print(df_review.head())

                  user_id             business_id
0  hG7b0MtEbXx5QzbzE6C_VA  ujmEBvifdJM6h6RLv4wQIg
1  yXQM5uF2jS6es16SJzNHfg  NZnhc2sEQy3RmzKTZnqtwQ
2  n6-Gk65cPZL6Uz8qRm3NYw  WTqjgwHlXbSFevF32_DJVw
3  dacAIZ6fTM6mqwW5uxkskg  ikCg8xy5JIg_NGPx-MSIDA
4  ssoyf2_x0EQMed6fgHeMyQ  b1b1eb3uo-w561D0ZfCEiQ


In [10]:
rec.mostpopular(df_review,10)

,business_id,score
0,4JNXUYY8wbaaDmk3BPzlWw,8570
1,RESDUcs7fIiihp38-d6_6g,8568
2,K7lWdNUhCbcnEvI0NhGewg,6887
3,f4x1YBxkLrZg652xt2KR5g,5847
4,cYwJA2A6I12KNkm2rtXd5g,5575
5,DkYS3arLOhA8si5uUEmHOw,5206
6,2weQS-RnoOBhb1KsHKyoSQ,4534
7,5LNZ67Yw9RD6nf4_UhXOjw,4522
8,iCQpiavjjPzJ5_3gPD5Ebg,4351
9,SMPbvZLSMMb7KU76YNYMGg,4350


In [11]:
df_review=None
tmp_rec_list=_

In [12]:

df_business=pd.read_feather("../data/business.feather",columns=['business_id','categories'],use_threads=3)
tmp_rec_list=pd.merge(tmp_rec_list,df_business,left_on='business_id',right_on='business_id').drop_duplicates().reset_index(drop=True)
df_business=None
tmp_rec_list

,business_id,score,categories
0,4JNXUYY8wbaaDmk3BPzlWw,8570,"Food, French, Breakfast & Brunch, Restaurants,..."
1,RESDUcs7fIiihp38-d6_6g,8568,"Restaurants, Buffets, Sandwiches, Food, Breakf..."
2,K7lWdNUhCbcnEvI0NhGewg,6887,"Buffets, Breakfast & Brunch, Restaurants"
3,f4x1YBxkLrZg652xt2KR5g,5847,"Breakfast & Brunch, Food, Restaurants, America..."
4,cYwJA2A6I12KNkm2rtXd5g,5575,"Restaurants, Burgers, American (Traditional)"
5,DkYS3arLOhA8si5uUEmHOw,5206,"Food Delivery Services, Salad, Sandwiches, Sou..."
6,2weQS-RnoOBhb1KsHKyoSQ,4534,"Beauty & Spas, Food, Event Planning & Services..."
7,5LNZ67Yw9RD6nf4_UhXOjw,4522,"Casinos, Hotels, Event Planning & Services, Re..."
8,iCQpiavjjPzJ5_3gPD5Ebg,4351,"Pizza, Restaurants"
9,SMPbvZLSMMb7KU76YNYMGg,4350,"Hotels & Travel, Arts & Entertainment, Hotels,..."


In [36]:
for i in range(len(tmp_rec_list)):
    print(tmp_rec_list['categories'].iloc[i])

Food, French, Breakfast & Brunch, Restaurants, Steakhouses
Restaurants, Buffets, Sandwiches, Food, Breakfast & Brunch
Buffets, Breakfast & Brunch, Restaurants
Breakfast & Brunch, Food, Restaurants, American (New)
Restaurants, Burgers, American (Traditional)
Food Delivery Services, Salad, Sandwiches, Soup, Food, Event Planning & Services, American (New), Restaurants, Caterers, Wraps
Beauty & Spas, Food, Event Planning & Services, Hotels, Buffets, Casinos, Day Spas, Restaurants, Arts & Entertainment, Hotels & Travel
Casinos, Hotels, Event Planning & Services, Restaurants, Arts & Entertainment, Hotels & Travel
Pizza, Restaurants
Hotels & Travel, Arts & Entertainment, Hotels, Event Planning & Services, Resorts, Casinos


# Categorias relevantes

In [6]:
Lambda = 0.5
Beta = 0.5

In [11]:
# lista temporária e lista final
rec_list=pd.DataFrame(columns=['business_id','score'])

k=len(tmp_rec_list) ## size of rec list
for i in range(k):
    print(i)
    for index,row in tmp_rec_list.iterrows():
        business_id=row['business_id']
        score=row['score']
        print(index,business_id,score)


0
1
2
3
4
5
6
7
8
9
0 4JNXUYY8wbaaDmk3BPzlWw 8570
1 RESDUcs7fIiihp38-d6_6g 8568
2 K7lWdNUhCbcnEvI0NhGewg 6887
3 f4x1YBxkLrZg652xt2KR5g 5847
4 cYwJA2A6I12KNkm2rtXd5g 5575
5 DkYS3arLOhA8si5uUEmHOw 5206
6 2weQS-RnoOBhb1KsHKyoSQ 4534
7 5LNZ67Yw9RD6nf4_UhXOjw 4522
8 iCQpiavjjPzJ5_3gPD5Ebg 4351
9 SMPbvZLSMMb7KU76YNYMGg 4350


In [25]:
def shortest_path_length(category1,category2):
    spl=0
    if category1 == category2:
       spl=0
    else:
        
        
def category_dis_sim(category1,category2):
    dissim=0.0
    spd=shortest_path_length(category1,category2)
    sim = 1.0 / (1.0 + spd)
    dissim=1.0-sim
    return dissim
    

def min_dist_to_list_cat(business,rec_list):
    min_dissim=1.0
    rec_list_size=len(rec_list)
    if rec_list_size==0:
        min_dissim=1.0
    else:
        for index,row in rec_list.iterrows():
            local_min_distance=1
            cur_distance=0
            for category1 in business['categories'].split(','):
                for category2 in row['categories'].split(','):
                    cur_distance=category_dis_sim(category1,category2)
                    local_min_distance=min(local_min_distance,cur_distance)
            min_dissim=min(min_dissim,local_min_distance)
    return min_dissim
        
def objective_ild(business_id,rec_list):
    # /*(1-_cfg._divWeight) * relevance + */_cfg._divWeight * diversity; where _cfg._divWeight = 0.9
    ret_val= 0.0
    diversity=0.0
    div_weight=0.9
    ret_val = div_weight*diversity #+ (1-div_weight)*relevance
    return ret_val
min_dist_to_list_cat(tmp_rec_list.iloc[0],tmp_rec_list)
# for index,row in tmp_rec_list.iterrows():
#     business_id=row['business_id']
#     score=row['score']
#     print(index,business_id,score)

0 4JNXUYY8wbaaDmk3BPzlWw Food, French, Breakfast & Brunch, Restaurants, Steakhouses
1 RESDUcs7fIiihp38-d6_6g Restaurants, Buffets, Sandwiches, Food, Breakfast & Brunch
2 K7lWdNUhCbcnEvI0NhGewg Buffets, Breakfast & Brunch, Restaurants
3 f4x1YBxkLrZg652xt2KR5g Breakfast & Brunch, Food, Restaurants, American (New)
4 cYwJA2A6I12KNkm2rtXd5g Restaurants, Burgers, American (Traditional)
5 DkYS3arLOhA8si5uUEmHOw Food Delivery Services, Salad, Sandwiches, Soup, Food, Event Planning & Services, American (New), Restaurants, Caterers, Wraps
6 2weQS-RnoOBhb1KsHKyoSQ Beauty & Spas, Food, Event Planning & Services, Hotels, Buffets, Casinos, Day Spas, Restaurants, Arts & Entertainment, Hotels & Travel
7 5LNZ67Yw9RD6nf4_UhXOjw Casinos, Hotels, Event Planning & Services, Restaurants, Arts & Entertainment, Hotels & Travel
8 iCQpiavjjPzJ5_3gPD5Ebg Pizza, Restaurants
9 SMPbvZLSMMb7KU76YNYMGg Hotels & Travel, Arts & Entertainment, Hotels, Event Planning & Services, Resorts, Casinos


In [27]:
for a in tmp_rec_list.iloc[0]['categories']:
    print(a)

F
o
o
d
,
 
F
r
e
n
c
h
,
 
B
r
e
a
k
f
a
s
t
 
&
 
B
r
u
n
c
h
,
 
R
e
s
t
a
u
r
a
n
t
s
,
 
S
t
e
a
k
h
o
u
s
e
s


In [31]:
tmp_rec_list.iloc[0]['categories'].split(',')


False